In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()
spark

In [3]:

df = spark.read.parquet("s3://aws-logs-780494741284-us-east-1/dataset/project_graph_data")

In [4]:
df.createOrReplaceTempView("df")
df.printSchema()

root
 |-- source: string (nullable = true)
 |-- target: string (nullable = true)



In [5]:
df.take(5)

[Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.HB6611.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.SX1822.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.DK8883.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.287720.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.MG8541.COM/')]

# Combine all urls together

In [9]:
df1 = df.select(['target'])
df2 = df.select(['source'])



In [10]:
df1 = df1.selectExpr("target as urls")
df1.show()
df1.printSchema()

+-------------------+
|               urls|
+-------------------+
|   /WWW.HB6611.COM/|
|   /WWW.SX1822.COM/|
|   /WWW.DK8883.COM/|
|   /WWW.287720.COM/|
|   /WWW.MG8541.COM/|
|  /WWW.1382121.COM/|
|  /WWW.3615005.COM/|
|     /WWW.7760.NET/|
| /WWW.HG123777.COM/|
|  /WWW.CP85555.COM/|
|    /WWW.99662.COM/|
| /WWW.13300007.COM/|
|/WWW.XPJ005005.COM/|
|    /WWW.GWSSC.COM/|
|  /WWW.BMW6609.COM/|
|  /WWW.WXC7779.COM/|
|    /WWW.CR949.COM/|
|  /WWW.8899480.COM/|
|   /WWW.142VNS.COM/|
|  /WWW.DGD0000.COM/|
+-------------------+
only showing top 20 rows

root
 |-- urls: string (nullable = true)



In [11]:
df2 = df2.selectExpr("source as urls")
df2.show()
df2.printSchema()

+--------------------+
|                urls|
+--------------------+
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
|http//0.ssbet138....|
+--------------------+
only showing top 20 rows

root
 |-- urls: string (nullable = true)



In [12]:
df_c = df1.unionByName(df2)
df_c.printSchema()

root
 |-- urls: string (nullable = true)



In [ ]:
from operator import itemgetter

indexer = (df_c.select("urls").distinct()
    .rdd.map(itemgetter(0)).zipWithIndex()
    .toDF(["urls", "label"]))

df_c = df_c.join(indexer, ["urls"]).show()

In [ ]:
df_c

In [29]:
dic = dict(df_c.rdd.map(lambda x: (x['urls'], x['label'])).collect())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 16.0 failed 4 times, most recent failure: Lost task 0.3 in stage 16.0 (TID 715, ip-172-31-85-209.ec2.internal, executor 15): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1586808820885_0001/container_1586808820885_0001_01_000023/pyspark.zip/pyspark/sql/types.py", line 1514, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'label' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1586808820885_0001/container_1586808820885_0001_01_000023/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/mnt/yarn/usercache/hadoop/appcache/application_1586808820885_0001/

In [26]:
df_dic[:3]

[{'urls': '/WWW.HB6611.COM/'},
 {'urls': '/WWW.SX1822.COM/'},
 {'urls': '/WWW.DK8883.COM/'}]

In [23]:
df_c_dic.get('http//0.ssbet138.cn/WWW.XPJ80886.COM/')

AttributeError: 'list' object has no attribute 'get'

In [ ]:
def url_node(s):
    node = 
    return node

In [ ]:
graph_edge = df1.map(url_node)